In [3]:
import pandas as pd
import numpy as np
import helpers as dsfh
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.impute import KNNImputer
import statsmodels.api as sm #statsmodels is used for creating linear regression models
from sklearn.model_selection import train_test_split #creating testing and training data
from sklearn.metrics import roc_auc_score #evaluate model performance
import predhelp as ph

In [4]:
ph.ppi_clean("/Users/schuler/Documents/GitHub.nosync/dsfgp/data/Producer_price_indices.csv")

KeyError: 2

In [8]:
ph.ppi_clean()

AttributeError: module 'predhelp' has no attribute 'ppi_clean'